In [ ]:
import pandas as pd
import re
from rapidfuzz import process
from fuzzywuzzy import fuzz
import ipywidgets as widgets
from IPython.display import display, HTML

# Load the data
file_path = r'S:\Projekte\FG12_NRZ_MMR\A_Mitarbeiter\Oleksandr Kolyvushko\Reports\Temp_einseder\all_time_Einsender_list.xlsx'
df = pd.read_excel(file_path)

# Convert 'Plz' column to numeric
df['Plz'] = pd.to_numeric(df['Plz'], errors='coerce').astype('Int64')

# Function to highlight search term
def highlight_search_term(text, search_term, color):
    regex = re.compile(re.escape(search_term), re.IGNORECASE)
    return regex.sub(lambda match: f'<mark style="background-color: {color};">{match.group(0)}</mark>', text)

# Function to highlight partial match
def highlight_partial_match(text, search_term, color):
    best_match = process.extractOne(search_term, [text])
    if best_match and best_match[1] > 60:
        return highlight_search_term(text, best_match[0], color)
    return text

# Fuzzy search function
def fuzzy_search_optimized(search_term, df, columns):
    df['combined'] = df[columns].apply(lambda x: ' '.join(x.astype(str)), axis=1)
    matches = process.extract(search_term, df['combined'], limit=100)
    matching_indices = [match[2] for match in matches]
    result_df = df.loc[matching_indices, columnsshown]
    
    for column in columns:
        result_df[column] = result_df[column].astype(str).apply(
            lambda x: highlight_search_term(x, search_term, 'yellow') 
                      if re.search(re.escape(search_term), x, re.IGNORECASE) 
                      else x
        )
    
    for column in columns:
        result_df[column] = result_df[column].astype(str).apply(
            lambda x: highlight_partial_match(x, search_term, 'orange') 
                      if fuzz.partial_ratio(search_term.lower(), x.lower()) > 60 
                      else x
        )
    
    display(HTML(result_df.to_html(index=False, escape=False)))

# Create a text input widget
search = widgets.Text(
    value='',
    placeholder='Type a search term',
    description='Search:',
    disabled=False,
    layout=widgets.Layout(width='400px')
)
search.style.description_width = 'initial'

# Columns to be shown
columnsshown = [
    'IdEinsender', 'Anschrift1', 'Anschrift2', 'Anschrift3', 
    'Strasse', 'Plz', 'Ort', 'Telefon', 'eMail', 'combined', 'EinKurz', 'CreatedAt'
]

# Set up the interactive display
output = widgets.Output()

def on_search_change(change):
    with output:
        output.clear_output()
        fuzzy_search_optimized(change['new'], df, ['EinKurz'])

# Add event listener to the text input
search.observe(on_search_change, names='value')

# Display the widgets
display(search, output)
